In [225]:
%%bash --out TOKEN
# Step 1: Authenticate and extract nonce using jq
nonce=$(curl -s 'http://localhost:7070/auth/authenticate' \
  -H 'Accept: application/json, text/plain, */*' \
  -H 'Accept-Language: en-GB,en-US;q=0.9,en;q=0.8' \
  -H 'Connection: keep-alive' \
  -H 'Content-Type: application/json' \
  -H 'Origin: http://localhost:3020' \
  -H 'Referer: http://localhost:3020/' \
  -H 'Sec-Fetch-Dest: empty' \
  -H 'Sec-Fetch-Mode: cors' \
  -H 'Sec-Fetch-Site: same-site' \
  -H 'User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36' \
  -H 'sec-ch-ua: "Chromium";v="130", "Google Chrome";v="130", "Not?A_Brand";v="99"' \
  -H 'sec-ch-ua-mobile: ?0' \
  -H 'sec-ch-ua-platform: "macOS"' \
  --data-raw '{"username":"k.mweene","password":"test"}' | jq -r '.nonce')

# Step 2: Use the extracted nonce in the verifyCode request
TOKEN=$(curl -s 'http://localhost:7070/auth/verifyCode' \
  -H 'Accept: application/json, text/plain, */*' \
  -H 'Accept-Language: en-GB,en-US;q=0.9,en;q=0.8' \
  -H 'Connection: keep-alive' \
  -H 'Content-Type: application/json' \
  -H 'Origin: http://localhost:3020' \
  -H 'Referer: http://localhost:3020/' \
  -H 'Sec-Fetch-Dest: empty' \
  -H 'Sec-Fetch-Mode: cors' \
  -H 'Sec-Fetch-Site: same-site' \
  -H 'User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36' \
  -H 'sec-ch-ua: "Chromium";v="130", "Google Chrome";v="130", "Not?A_Brand";v="99"' \
  -H 'sec-ch-ua-mobile: ?0' \
  -H 'sec-ch-ua-platform: "macOS"' \
  --data-raw "{\"code\":\"000000\",\"nonce\":\"$nonce\"}" | jq -r '.token')

echo "$TOKEN"

# Create an "event"

In [226]:
%%bash -s "$TOKEN" --out id

TOKEN=$1
response_data=$(curl --request POST \
  --url "http://localhost:7070/graphql" \
  --header 'Authorization: Bearer '$TOKEN \
  --header 'Content-Type: application/json' \
  -d '{
      "query": "mutation CreateEvent($input: EventInput!) { createEvent(event: $input) { id type createdAt updatedAt actions { ... on CreateAction { type createdAt createdBy data { id value } } } } }",
      "variables": { "input": { "type": "tennis-club-membership" } }
    }')

id=$(echo "$response_data" | jq '.data.createEvent.id')
echo "$id"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3609  100  3331  100   278   588k  50289 --:--:-- --:--:-- --:--:--  704k


In [227]:
%%bash -s "$id" "$TOKEN"

id=$(echo "$1" | xargs)
TOKEN=$2

echo "Event ID: '$id'"
echo "Token: $TOKEN"

response_data=$(curl --request POST \
  --url "http://localhost:7070/graphql" \
  --header 'Authorization: Bearer '$TOKEN \
  --header 'Content-Type: application/json' \
  -d '{
      "query": "mutation DeclareEvent($eventId: ID!, $input: DeclareActionInput!) { declareEvent(eventId: $eventId, input: $input) { id type createdAt updatedAt actions { ... on DeclareAction { type createdAt createdBy data { id value } } } } }",
      "variables": {
        "eventId": "'${id}'",
        "input": {
          "data": [
            { "id": "applicant.firstname", "value": "Riku" },
            { "id": "applicant.surname", "value": "Rouvila" },
            { "id": "applicant.dob", "value": "2024-11-27T11:56:20.781Z" }
          ]
        }
      }
    }')
echo "$response_data" | jq

Event ID: 'null'
Token: eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCJ9.eyJzY29wZSI6WyJyZWdpc3RlciIsInBlcmZvcm1hbmNlIiwiY2VydGlmeSIsImRlbW8iXSwiaWF0IjoxNzMyNzE0OTM4LCJleHAiOjE3MzMzMTk3MzgsImF1ZCI6WyJvcGVuY3J2czphdXRoLXVzZXIiLCJvcGVuY3J2czp1c2VyLW1nbnQtdXNlciIsIm9wZW5jcnZzOmhlYXJ0aC11c2VyIiwib3BlbmNydnM6Z2F0ZXdheS11c2VyIiwib3BlbmNydnM6bm90aWZpY2F0aW9uLXVzZXIiLCJvcGVuY3J2czp3b3JrZmxvdy11c2VyIiwib3BlbmNydnM6c2VhcmNoLXVzZXIiLCJvcGVuY3J2czptZXRyaWNzLXVzZXIiLCJvcGVuY3J2czpjb3VudHJ5Y29uZmlnLXVzZXIiLCJvcGVuY3J2czp3ZWJob29rcy11c2VyIiwib3BlbmNydnM6Y29uZmlnLXVzZXIiLCJvcGVuY3J2czpkb2N1bWVudHMtdXNlciJdLCJpc3MiOiJvcGVuY3J2czphdXRoLXNlcnZpY2UiLCJzdWIiOiI2NzMzMzA5ODI3Yjk3ZTY0ODM4NzcxODgifQ.wSLoqLpBKjLwVydLkP7vZCb8sDKe4y-DDNMWR-o7hViq87pLuBLUtEEyanqD-VMmHr5nIV6A8aALwR5ydN1b8aokYRDmu5XXSxGp0yylQ3of1GjgMq3IaDR0q483LYgOFdMmxyszw22C9RMa4n92oc3kJchjTsljZZVdWQIk6kNKDJ4f0xBpsN_z4K3PZI35BRF-cTDua5y9pP8NdsIwEQB08z8PNhu0ZMglqUYibfPgdD-0Pn8UqY56L_ZRq6h8IRbsEzNA_u2u5jkAzmtICWYyXAJbwAcRpJOZgJcSTbMzvqWIXBAnD1z4WsScDTj1xXRKGCQA

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3902  100  3331  100   571   619k   106k --:--:-- --:--:-- --:--:--  762k


In [230]:
%%bash -s "$TOKEN"

TOKEN=$1

response_data=$(curl --request POST \
  --url "http://localhost:7070/graphql" \
  --silent \
  --header 'Authorization: Bearer '$TOKEN \
  --header 'Content-Type: application/json' \
  -d '{
      "query": "query GetEventConfig { getEventConfig { id label { id defaultMessage description } actions { label { id defaultMessage description } type forms { label { id defaultMessage description } version { id label { id defaultMessage description } } active pages { id title { id defaultMessage description } fields { ... on TextField { id required label { id defaultMessage description } options { maxLength } } ... on DateField { id required label { id defaultMessage description } options { notice { id defaultMessage description } } } ... on ParagraphField { id required label { id defaultMessage description } } } } } } } }"
    }')
echo "$response_data" | jq

{
  "data": {
    "getEventConfig": [
      {
        "id": "TENNIS_CLUB_MEMBERSHIP",
        "label": {
          "id": "event.tennis-club-membership.label",
          "defaultMessage": "Tennis club membership application",
          "description": "This is what this event is referred as in the system"
        },
        "actions": [
          {
            "label": {
              "id": "event.tennis-club-membership.action.declare.label",
              "defaultMessage": "Send an application",
              "description": "This is shown as the action name anywhere the user can trigger the action from"
            },
            "type": "DECLARE",
            "forms": [
              {
                "label": {
                  "id": "event.tennis-club-membership.action.declare.form.label",
                  "defaultMessage": "Tennis club membership application",
                  "description": "This is what this form is referred as in the system"
                },
                